In [1]:
import json
import networkx as nx

from utils.get_anchor_list import *
from utils.greedy import *

In [2]:
folder = "./Example_data/nG50/"
n_clust = 8
nodes = 50
tlim = 6
sol_n = 0

q_con = 0.4
q_mod = 0.7


In [3]:
def read_input_ori(extantFNAME):    
    extant_edges = []
    with open(extantFNAME) as f:
        for line in f:
            li = [x.strip() for x in line.split()]
            assert(len(li) == 2)
            extant_edges.append((int(li[0]),int(li[1])))
    
    extant = nx.Graph()
    extant.add_edges_from(extant_edges)
    return extant

def read_input(extantFNAME):
    with open(extantFNAME, "r") as f:
        datastruct = json.load(f)
        
    # no constraint, original ILP
    first = nx.Graph() 
    for idx, x in enumerate(datastruct['start_nodes']):
        first.add_node(idx + 1)   
    first_edges = [tuple(x) for x in datastruct['first_graph']]

    # possibly one node and no connection, which gives []           
    if first_edges != []:
        first.add_edges_from(first_edges)
    else:
        pass
    
    # last
    last = nx.Graph()
    last_edges = [tuple(x) for x in datastruct['last_graph']]

    for idx, x in enumerate(datastruct['last_nodes']):
        last.add_node(idx + 1)
    last.add_edges_from(last_edges)
    
    return last_edges, last, first_edges, first

In [4]:
ARlist_ori = []
extant_name = folder + "nx_run=1_qmod="+ str(q_mod) + "_qcon=" + str(q_con) + "_n=" + str(nodes)
extant = read_input_ori(extant_name)

# DCH-ILP

In [5]:
for group in range(n_clust - 1, -1, -1):
    result_name = folder + "ILP_nogreedy_group" + str(group) + "_tlim="+ str(tlim) +"_solution" + str(sol_n) + ".txt"
    file_name = folder + "group" + str(group) + ".json"
    with open(file_name, "r") as fp:
        data_dict = json.load(fp)

    n = len(data_dict['last_nodes'])
    # start nodes can be 1, 2, >2, except the first one, which has the number of start cluster 0
    f = len(data_dict['start_nodes'])

    assert f == 2 or f == 0
    if f == 0:
        f = 2

    # create solution object
    soln = read_file(result_name, create_ds(n))   
    ARlist = get_anchor_list(n, f, soln)
    for a, r in ARlist:
        
        ARlist_ori.append([data_dict['last_nodes'][a-1] + 1, data_dict['last_nodes'][r-1] + 1]) 
    if len(data_dict['start_nodes']) != 0:
        ARlist_ori.append([x + 1 for x in data_dict['start_nodes']])
        ARlist.append([1, 2])
# have problem with concatenate, try 1,0 first
graphlist = get_all_networks(extant, ARlist_ori) 

number of graphs: 49 , number of anchor remove pairs: 48


In [6]:
lkl, log_lkl = compute_totlkl(graphlist[:-1], ARlist_ori, q_mod = 0.7, q_con = 0.4)
print("log likelihood value of DCH-ILP", log_lkl)

log likelihood value of DCH-ILP -101.41129794888047


# Greedy

In [7]:
edges = []

adj = np.zeros((nodes,nodes))
with open(folder + "nx_run=1_qmod="+str(q_mod) +"_qcon="+ str(q_con) + "_n="+str(nodes), mode="r") as fp:
    for count, line in enumerate(fp):
        # start from one
        edges.append(tuple([eval(x) for x in line[:-1].split(" ")]))
        adj[edges[-1][0] - 1, edges[-1][1] - 1] = adj[edges[-1][1] - 1, edges[-1][0] - 1] = 1
# node start from 1
g = nx.Graph()
g.add_nodes_from(np.arange(n))
g.add_edges_from(edges)

In [8]:
ARlist, last_node = dmc_delorean(g,q_mod,q_con)
last_pair = (last_node[0], ARlist[-1])
# leave two nodes in the end rather than one nodes
g_list = get_all_networks(g, ARlist)

number of graphs: 51 , number of anchor remove pairs: 50


In [9]:
lkl, log_lkl = compute_totlkl(g_list[:-1], ARlist, q_mod = q_mod, q_con = q_con)
print("log likelihood value of Greedy", log_lkl)

log likelihood value of Greedy -110.36561480384182
